<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/01_Energy_Convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Jupyter notebook

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) (for the quantum chemistry)
  * [py3DMol](https://3dmol.csb.pitt.edu/) for visualizing the molecule
  * [plotly](https://plotly.com/python/) and kaleido for plotting

In [1]:
%pip install -q pyscf py3DMol plotly kaleido

In [2]:
from pyscf import gto, scf, dft, mp, cc
import py3Dmol
import plotly.express as px

# Setting up our system

- We initialize the molecular (or solid) PySCF object with coordinates, symmetry, basis, spin, and charge information
- We can check that things look right with [py3DMol](https://3dmol.csb.pitt.edu/). Use your mouse to move the molecular around!



In [3]:
# Experimental geometry of gas-phase water
# Ref: https://cccbdb.nist.gov/expgeom2x.asp
mol_xyz = """O        0.0000   0.0000   0.1173
             H        0.0000   0.7572	 -0.4692
             H        0.0000  -0.7572	 -0.4692"""
mol = gto.M(atom=mol_xyz, basis="def2tzvp", verbose=4) # TODO: add symmetry, spin, and charge but say they are handled by default

System: uname_result(system='Linux', node='16878c4dceaf', release='5.4.188+', version='#1 SMP Sun Apr 24 10:03:06 PDT 2022', machine='x86_64', processor='x86_64')  Threads 2
Python 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
numpy 1.21.6  scipy 1.4.1
Date: Fri Jun  3 18:17:40 2022
PySCF version 2.0.1
PySCF path  /usr/local/lib/python3.7/dist-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 O      0.000000000000   0.000000000000   0.117300000000 AA    0.000000000000   0.000000000000   0.221664874411 Bohr
[INPUT]  2 H      0.000000000000   0.757200000000  -0.469200000000 AA    0.000000000000   1.430900621521  -0.886659497646 Bohr
[INPUT]  3 H      0.000000000000  -0.757200000000  -0.469200000000 AA    0.000000000000  -1.430900621521  -0.886659497646 Bohr

nuclear repulsion = 9.1895

In [4]:
xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Short survey of quantum chemistry methods

## [Hartree-Fock](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method)


* Hatree-Fock (HF) is the starting point of the most of quantum chemistry
* We variationally optimize the orbitals for a single [Slater determinint](https://en.wikipedia.org/wiki/Slater_determinant)
* Working in the basis of atom-centered basis function we solve the [Roothaan-Hall](https://en.wikipedia.org/wiki/Roothaan_equations) equations:

$\textbf{FC} = \textbf{SC} \epsilon$

* $\textbf{F}$ is the [Fock matrix]()
* $\textbf{C}$ is the molecular orbital coefficient matrix
* $\textbf{S}$ is the atomic orbital overlap matrix
* $\epsilon$ is the vector of molecular orbital energies

See the PySCF [user guide](https://pyscf.org/user/scf.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/scf) for more info.



In [5]:
mymf = scf.RHF(mol).run()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /content/tmpetlqsile
max_memory 4000 MB (current use 210 MB)
Set gradient conv threshold to 3.16228e-05
init E= -75.8829473413812
  HOMO = -0.479044698163315  LUMO = 0.072228440089738
cycle= 1 E= -76.022968316524  delta_E= -0.14  |g|= 0.438  |ddm|= 0.926
  HOMO = -0.432554692380851  LUMO = 0.125726455074504
cycle= 2 E= -76.0491789648116  delta_E= -0.0262  |g|= 0.25  |ddm|= 0.308
  HOMO = -0.519086888918856  LUMO = 0.123542897750612
cycle= 3 E= -76.0587091509589  delta_E= -0.00953  |g|= 0.0381  |ddm|= 0.115
  HOMO = -0.506929277770514  LUMO = 0.12751188697308
cycle= 4 E= -76.0589890103909  delta_E= -0.00028  |g|= 0.00589  |ddm|= 0.0206
  HOMO = -0.50780496455

## [Density Functional Theory](https://en.wikipedia.org/wiki/Density_functional_theory)

* In Density Functional Theory (DFT), the electron density of a reference noninteracting system is used to represent the density of the true interacting system.
* The formulation resembles HF with a different effective Fock potential.
* This effective potential depends on the density functional approximation which is chosen by the user.
* PySCF gives users the access to a large number of functionals through the [libxc](https://tddft.org/programs/libxc/) and [xcfun](https://github.com/dftlibs/xcfun) libraries.

See the PySCF [user guide](https://pyscf.org/user/dft.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/dft) for more info.

In [6]:
myrks = dft.RKS(mol, xc="PBE").run()



******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /content/tmpru1i6_f9
max_memory 4000 MB (current use 222 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = PBE
    J. P. Perdew, K. Burke, and M. Ernzerhof, Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof, Phys. Rev. Lett. 78, 1396 (1997)
    J. P. Perdew, K. Burke, and M. Ernzerhof, Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof, Phys. Rev. Lett. 78, 1396 (1997)
radial grids: 
    Treutler-Ahlrichs [JCP 102, 346 (1995); DOI:10.1063/1.469408] (M4) radial grids
    
becke partition: Becke, JCP 88

## [Møller–Plesset perturbation theory](https://en.wikipedia.org/wiki/M%C3%B8ller%E2%80%93Plesset_perturbation_theory)

* Perturbative corrections to the Hartree-Fock approximation.

See the PySCF [user guide](https://pyscf.org/user/mp.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/mp) for more info.

In [7]:
mymp2 = mp.MP2(mymf).run()


******** <class 'pyscf.mp.mp2.MP2'> ********
nocc = 5, nmo = 43
max_memory 4000 MB (current use 292 MB)
E(MP2) = -76.3316077548634  E_corr = -0.272602994286647


## [Coupled Cluster](https://en.wikipedia.org/wiki/Coupled_cluster)

* Perturbative method that improves on the Hartree-Fock approximation.
* Coupled Cluster Singles and Doubles (CCSD) includes single and double excitation on top of the HF wave function.
* Accuracy can be improved by including triples perturbatively (CCSD(T)).
* Non-variational, but size extensive description of ground states. For excited states, see [EOM-CCSD]().

See the PySCF [user guide](https://pyscf.org/user/cc.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/cc) for more info.

In [8]:
mycc = cc.CCSD(mymf).run()


******** <class 'pyscf.cc.ccsd.CCSD'> ********
CC2 = 0
CCSD nocc = 5, nmo = 43
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 292 MB)
Init t2, MP2 energy = -76.3316077434556  E_corr(MP2) -0.272602982878812
Init E_corr(CCSD) = -0.272602982878943
cycle = 1  E_corr(CCSD) = -0.274027187984409  dE = -0.00142420511  norm(t1,t2) = 0.0246931
cycle = 2  E_corr(CCSD) = -0.278753761838253  dE = -0.00472657385  norm(t1,t2) = 0.00806209
cycle = 3  E_corr(CCSD) = -0.279276820293764  dE = -0.000523058456  norm(t1,t2) = 0.0027462
cycle = 4  E_corr(CCSD) = -0.279629076314956  dE = -0.000352256021  norm(t1,t2) = 0.000899687
cycle = 5  E_corr(CCSD) = -0.279638117060696  dE = -9.04074574e-06  norm(t1,t2) = 0.000187256
cycle = 6  E_corr(CCSD) = -0.279622033353006  dE = 1.60837077e-05  norm(t1,t2) = 5.34116e-05
cycle = 7  E_corr(CCSD) = -0.2796247587977  dE = -2.72544469e-06  norm(t1,t2) = 

In [9]:
e_ccsd_t = mycc.ccsd_t()

CCSD(T) correction = -0.00703817132754736


# Analysis

* TODO

In [11]:
# Collect data
methods = ["HF", "MP2", "CCSD", "CCSD(T)"]
energies = [mymf.e_tot, mymp2.e_tot, mycc.e_tot, mycc.e_tot + e_ccsd_t]

# Plotting
fig = px.line(x=methods, y=energies, title="Jacob's Ladder", markers=True)
fig.update_layout(xaxis_title="Method", yaxis_title="Energy (Ha)")
fig.update_traces(marker_size=12)
fig.show() # It's interactive!